<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2022-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Discontinuous Galerkin Methods (DG Methods) <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Ecuacion de Poisson <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

In [51]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
#import netgen.gui

# Domain geometry
geo = unit_square
# Generate mesh with meshsize
hh = .1
mesh = Mesh(geo.GenerateMesh(maxh=hh))

# Manufactured solution
uexact = CoefficientFunction( 16*x*(1-x)*y*(1-y) )
qexact = CoefficientFunction((-16*(1-2*x)*y*(1-y), -16*(1-2*y)*x*(1-x)  ))
# Source according 
f      = 32*y*(1-y)+32*x*(1-x)
Draw(uexact, mesh, 'solucion exacta u ')
Draw(qexact, mesh, 'solucion exacta q')
#print(mesh.GetBoundaries())

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# DG method

Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\Delta u & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & 0 & \mbox{sobre} & \partial \Omega 
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $q$, y $a = c^{-1}$
\begin{equation}
\begin{array}{rclll}
\displaystyle  q & = & -\nabla u & \mbox{en} & \Omega \\
\displaystyle
\mbox{div}( q ) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & 0 & \mbox{sobre} & \partial \Omega 
\end{array}
\end{equation}


Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(q,u)\in V_h\times W_h$

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{\Omega} q_h \cdot r\,dx - \int_{\Omega} u_h\,\nabla_h\cdot r\,dx + \sum_{K\in\mathcal T_h}\int_{\partial K} \phi_{u} r\cdot n\,ds & = &
0 \\
\displaystyle
-\int_{\Omega} q_h \cdot \nabla_h v\,dx + \sum_{K\in \mathcal T_h}\int_{\partial K} v \phi_{q} \cdot n \,ds & = & 
\displaystyle
 \int_{\Omega} f v\,dx
\end{array}
\end{equation}

Espacios de Galerkin discontinuo:
\begin{equation}
V_h = \{r\in L^{1}(\Omega)^{d}: (r_i)|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h,\,i=1,...,d\},\quad W_h = \{v\in L^{1}(\Omega): v|_{K}\in \mathcal P_{k},\,\forall K\in \mathcal T_h\}
\end{equation}

# Formulacion Mixta Local Discontinuous Galerkin

\begin{equation}
\phi_{u}(u_h) = \begin{cases}
\{u_h\} - \beta\cdot [u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
0 ,&\quad\text{sobre } \mathcal F_h^{\partial}, 
\end{cases}
\end{equation}
\begin{equation}
\phi_{q}(u_h,q_h) = \begin{cases}
\{q_h\} + \beta [q_h] - \eta_{F} h_{F}^{-1}[u_h],&\quad\text{sobre } \mathcal F_h^{i}, \\
q_h - \eta_F h_F^{-1} u_h n ,&\quad\text{sobre } \mathcal F_h^{\partial}, 
\end{cases}
\end{equation}


In [53]:
# Mixed form
order = 3
Wh = L2(mesh, order=order, dgjumps=True)
Vh = VectorL2(mesh, order=order, dgjumps=True)
fes = FESpace([Vh,Wh] )

# Trial and test functions
q, u = fes.TrialFunction()
r, v = fes.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_v = (v-v.Other())*n
jump_q = (q-q.Other())*n
jump_r = (r-r.Other())*n

mean_u = 0.5*(u+u.Other())
mean_v = 0.5*(v+v.Other())
mean_q = 0.5*(q+q.Other())
mean_r = 0.5*(r+r.Other())

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size
# LDG fluxes
phi_u_i = mean_u - beta*jump_u
phi_q_i = mean_q + beta*jump_q - eta/h*jump_u 
phi_u_b = 0
phi_q_b = q - eta/h*u*n 

dS = dx(element_boundary=True)
a_dg = BilinearForm(fes)
# first equation
a_dg += (q*r - u*div(r) )* dx 
a_dg += phi_u_b*(r*n)*ds(skeleton=True)
a_dg += phi_u_i*jump_r*dx(skeleton=True)
# second equation
a_dg += -(q*grad(v))* dx 
a_dg += (phi_q_i*jump_v)*dx(skeleton=True)
a_dg += (phi_q_b*v*n)*ds(skeleton=True)
a_dg.Assemble()

fdg = LinearForm(fes)
fdg += f*v*dx 
fdg.Assemble()

gf = GridFunction(fes)
gfq, gfu = gf.components
gf.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)
Draw (gfq)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

# Formulacion primal e Non-symmetric Interior Penalty Method


\begin{equation}
a_{h}(u_h,v) = \int_{\Omega}\nabla_h u_h\cdot\nabla v_h + \int_{\mathcal F_h} \eta_F h_F^{-1}[u_h]\cdot[ v] +[u_h]\cdot\{\nabla_h v\} - \{\nabla_h u_h\}[v]
\end{equation}
para $u_h,v\in W_h = \{v\in L^{1}(\Omega):\,v|_{K}\in \mathcal P_k,\,\forall K\in \mathcal T_h\}$

In [56]:
# NIPG
order = 4
Wh = L2(mesh, order=order, dgjumps=True)
u,v = Wh.TnT()



jumpn_u = n*(u-u.Other())
jumpn_v = n*(v-v.Other())
n = specialcf.normal(2)
mean_du = 0.5 * (grad(u)+grad(u.Other()))
mean_dv = 0.5 * (grad(v)+grad(v.Other()))

# jump_u = u-u.Other()
# jump_v = v-v.Other()
# mean_dudn = 0.5*n * (grad(u)+grad(u.Other()))
# mean_dvdn = 0.5*n * (grad(v)+grad(v.Other()))


etaF = 4*(order+1)**2
alpha = etaF
h = specialcf.mesh_size
ah = BilinearForm(Wh)
ah += grad(u)*grad(v) * dx
ah += etaF/h*jumpn_u*jumpn_v*dx(skeleton=True)
ah += jumpn_u*mean_dv*dx(skeleton=True)
ah += -mean_du*jumpn_v*dx(skeleton=True)
ah += etaF/h*u*n*v*n*ds(skeleton=True)
ah += u*n*grad(v)*ds(skeleton=True)
ah += -grad(u)*v*n*ds(skeleton=True)

ah.Assemble()

lf = LinearForm(f*v*dx).Assemble()

gfu = GridFunction(Wh, name="uDG")
gfu.vec.data = ah.mat.Inverse() * lf.vec
Draw (gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene